In [ ]:
# Do some rudementary offline particle tracking based on the mean 2d cross section of u and w velocities.
# release a number of particles at different heights near the western edge of the domain. 
# heights every 500m

# lbuchart
# January 15, 2023

In [ ]:
# imports
import numpy as np
import json
import os

from scipy.spatial import KDTree, distance
from wrf import getvar, extract_times
from file_funcs import setup_script
from context import json_dir
from netCDF4 import Dataset
from context import name_dir, script_dir

In [ ]:
class ParticleTracker:
    """
    First-Order offline particle tracking in 2D. Using WRF output.
    Processed using WRF-Python to create a experiment mean cross section
    
    Advection (callable): Function to advect the particles in space
    Nearest (callable): find nearest velocity point to the tracker
    setup_scripts (callable): setup files and path for WRF run (Compute Canada configured)
    WRF_proc (callable): function to process wrf output for use
    get_vel (callable): function to grab the velocities from WRF output
    x0z0 (tuple): initial position to release particle from
    u (float): horizontal velocity 
    w (float): vertical velocity
    time_step (float): time step we will march over 
    exp (string): name of experiment to process
    """
    
    def __init__(self, Advection, Nearest, setup_script, WRF_proc, get_vel, x0z0, u, w time_step, exp):
        
        self.Advection = Advection  # first order linear advection
        self.Nearest = Nearest  # find nearest velocity points to calculate with
        self.setup_script = setup_script  # function to setup wrf processing
        self.WRF_proc = WRF_proc  # function to process and load WRF winds
        self.get_vel = get_vel  # function to grab the velocities from wrf output
        
        self.x0z0 = x0z0  # position of particle on grid
        self.u = u  # horz velocity
        self.w = w  # vertical velocity
        
        self.time_step = time_step  # advection timestep
        self.exp = exp  # name of the experiment to process
          
    def Calculate(self):
        # calculate the initial state
        path, wrfin = setup_script(self.exp)
        U, W, nodes = WRF_proc(path, wrfin)
        index = Nearest(self.x0z0, nodes)
        self.u, self.w = get_vel(index, U, W) 
        
    def Update(self):
        self.x0z0 = Advection(self.timestep, self.x0z0, self.u, self.w)   
        
    def Advection(time_step, pos0, u, w):
        
        # linear advection in each direction
        
        x1 += pos0[0] + (u * time_step)
        z1 += pos0[1] + (w + time_step)
        
        
        return x1, z1
    
    def Nearest(pos, nodes):
        
        # use a linear interpolation to find u  and w
        # velocity from wrf interpolated output
        
        closest_index = distance.cdist(pos, nodes).argmin()
        
        return closest_index
            
    def setup_script(exp):
        path = name_dir + exp + "/output/"

        save_path = script_dir + "figures/" + exp
    
        if not os.path.exists(save_path):
            os.makedirs(save_path)
    
        # create list of file names (sorted) 
        all_files = sorted(os.listdir(path))

        # get wrfoutfiles
        file_start = "wrfout"
        # function to just grab the wrfout files from a given directroy
        relevant_files = []
        for file_name in all_files: 
            if file_name.startswith(file_start):
                relevant_files.append(file_name)

        # import all datasets
        wrfin = [Dataset(path+x) for x in relevant_files]
    
        return path, wrfin
    
    def WRF_proc(path, wrfin):
        
        # extract heights of all layers 
        heights = getvar(wrfin[0], "height_agl", units="m", meta=False)
        ys = heights[:, 0, 0] 

        # get grid resolution from our json dictionary
        with open(str(json_dir) + "config.json") as f:
            config = json.load(f)
        dx = config["grid_dimensions"]["dx"]
        ndx = config["grid_dimensions"]["ndx"]

        # create vector of distance
        horz_dist = np.arange(1, ndx+1, 1)
        for ii in range(0, len(horz_dist)):
            horz_dist[ii] = ii  * dx
            
        # make a (n, 2) array of every possible location on the grid by height and distance from edge
        nodes = []
        count = 0    
        for ii in range(0, len(ys)):
            for jj in range(0, len(horz_dist)):
                nodes[count, 0] = ys[ii]
                nodes[count, 1] = horz_dist[jj]
        
                count += 1
    
        # loop through files list and make our plots
        all_U = []
        all_W = []
    
        for ii in range(0, len(wrfin)):
            # import the file in a readable netcdf format
            ncfile = wrfin[ii]
    
            # get the time in datetime format
            ct = extract_times(ncfile, timeidx=0)
    
            # winds
            W = getvar(ncfile, "wa", units="m s-1",
                       meta=True)
            U = getvar(ncfile, "ua", units="m s-1",
                       meta=True)
        
            # take mean over the north south distance
            W = np.mean(W, axis=2)
            U = np.mean(U, axis=2)
        
            # concatenate over all time to get a mean picture
            WW = W.to_numpy()
            UU = U.to_numpy()
            if ii == 5:
                all_W = WW
                all_U = UU
            elif ii > 5: 
                all_W = np.dstack((all_W, WW))
                all_U = np.dstack((all_U, UU))
            
        final_U = np.mean(all_U, axis=2)
        final_W = np.mean(all_W, axis=2)
    
        return final_U, final_W, nodes
    
    def get_vel(index, U, W):
        # function to grab the actual velocity from the wrf output
        # based on the index values returned by the Nearest Function
        
        ii, jj = index
        
        u_use = U[ii, jj]
        w_use = W[ii, jj]
        
        return u_use, w_use

In [ ]:
# simulation parameters
# time
num_steps = 100
time_step = 1  # time step [ss]
wrf_dt = 300  # the "time step" of wrf output 5mins [s]

# release points
init_pos = [100, 1000]  # 100m from west edge, 1000m AGL

exp = "real" + "/" # name of the experiment you want to plot

## END USER INPUTS ##

# empty vector for particle positions
X = []
Z = []

In [ ]:
# initialize the particle tracking
pt = ParticleTracker(Advection, Nearest, setup_script, WRF_proc, get_vel, X, init_pos, time_step, exp)

X[0] = init_pos[0]
Z[0] = init_pos[1]
for ex in exp:
    path, wrfin = setup_script(exp)
    U, W, nodes = WRF_proc(path, wrfin)
    
    for ii in range(np.linspace(1, num_steps, time_step)):
        pt.Calculate()
        pt.Update()
        pos = pt.x0z0
        X[ii] = pos[0]
        Z[ii] = pos[1]